# Load libraries

In [1]:
from tensorflow import keras
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense, Flatten
from tensorflow.keras.layers import Embedding
from keras.models import Sequential
from tensorflow.keras import layers, models, losses, optimizers,callbacks

from keras.layers import Dense,LSTM
from keras.utils import pad_sequences


import pandas as pd
import numpy as np
import re

import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
stopwords = stopwords.words('english')

# load preprocessed data

In [2]:
df = pd.read_csv('preprocessed_data.csv')
df.shape

(158353, 6)

In [3]:
df['clean_lyrics'] = df['clean_lyrics'].astype(str)

# Create X and y feature

In [8]:
df = df.sample(frac=0.1)

X = df['clean_lyrics']

y = df['label']

# Vectorize X

In [9]:
# Limiting our tokenizers vocab size
max_words = 10000
 
    
# create the tokenizer
tokenizer = Tokenizer(num_words=max_words)


# Fit the tokenizer
tokenizer.fit_on_texts(X)


# Create the sequences for each sentence, basically turning each word into its index position
sequences = tokenizer.texts_to_sequences(X)


index_word = tokenizer.index_word


# # Limiting our sequencer to only include 300 words
max_length = 300


# # Convert the sequences to all be the same length of 300
X = pad_sequences(sequences, maxlen=max_length, padding='post')
print(X.shape)

(15835, 300)


# Split train and test sets

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=45)
X_train.shape,y_train.shape

((12668, 300), (12668,))

# Building LSTM nueral net 

In [11]:
# This creates the Neural Network
model = Sequential() 

# This embedding layer basically will automatically create the word2vec vectors based on your text data.
model.add( Embedding(max_words, 32, input_length=max_length) ) 

model.add(LSTM(50,return_sequences=True,dropout =0.2))
model.add(LSTM(50,dropout =0.2))
model.add(Dense(1,kernel_initializer='normal',  activation='linear'))
optimizer = optimizers.Adam(lr=0.003)
model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mse']) 

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 32)           320000    
                                                                 
 lstm (LSTM)                 (None, 300, 50)           16600     
                                                                 
 lstm_1 (LSTM)               (None, 50)                20200     
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 356,851
Trainable params: 356,851
Non-trainable params: 0
_________________________________________________________________


C:\Users\dip2l\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


# Training

In [12]:
callback = callbacks.EarlyStopping(monitor='val_mse',patience = 2,restore_best_weights=True)
hist = model.fit(X_train, y_train, 
                 validation_split=0.2,
                 epochs=1, batch_size=20,callbacks=[callback])

507/507 [==============================] - 123s 233ms/step - loss: 0.0653 - mse: 0.0653 - val_loss: 0.0615 - val_mse: 0.0615


In [14]:
type(model)

keras.engine.sequential.Sequential

In [15]:
#Store
model.save('LSTM_Valence_model.h5')

In [17]:
loaded_model = keras.models.load_model('LSTM_Valence_model.h5')

In [50]:
#load
loaded_model = tf.load_model('LSTM_Valence_model')

AttributeError: module 'tensorflow' has no attribute 'load_model'

In [54]:
print(type(loaded_model))

<class 'tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject'>


In [19]:
loaded_model.predict()

TypeError: Model.predict() missing 1 required positional argument: 'x'

In [52]:
loaded_model.predict()

AttributeError: '_UserObject' object has no attribute 'predict'

# Testing

In [11]:
mse= model.evaluate(X_test, y_test, verbose=0)[1]

print('Test mse with stacked LSTM:', mse)

AttributeError: '_UserObject' object has no attribute 'evaluate'